In [1]:
# Imports

import numpy as np
import pandas as pd
import math
import cv2
import time
from scipy import signal

In [2]:
###########################################################################
# CHANGE PATHS HERE DEPENDING ON WHERE YOU STORED THE VIDEO AND LOGS
###########################################################################

# Evaluation video path (download it by right-clicking on the video and saving as .mp4)
video_path = "evaluation-0-video.mp4"

# Get Evaluation logs
evaluation_file_path = "evaluation_job_logs/sim-trace/evaluation/0-iteration.csv"
evaluation_df = pd.read_csv(evaluation_file_path)

# Check content of evaluation logs
evaluation_df.head()

,episode,steps,X,Y,yaw,steer,throttle,action,reward,done,all_wheels_on_track,progress,closest_waypoint,track_len,tstamp,episode_status,pause_duration
0,0,1.0,-6.582955,-2.398817,165.716189,10.0,3.0,14,0.0,False,True,0.242192,1,57.862137,7.199,prepare,0.0
1,0,2.0,-6.582821,-2.398845,165.715203,10.0,3.0,14,1.0,False,True,0.241954,1,57.862137,7.265,in_progress,0.0
2,0,3.0,-6.592715,-2.394681,165.274801,-30.0,1.0,0,1.0,False,True,0.260304,1,57.862137,7.333,in_progress,0.0
3,0,4.0,-6.611892,-2.380354,163.349683,10.0,3.0,14,1.0,False,True,0.298546,1,57.862137,7.390,in_progress,0.0
4,0,5.0,-6.639581,-2.354663,159.316615,20.0,2.0,16,1.0,False,True,0.355908,2,57.862137,7.466,in_progress,0.0


In [21]:
class Simulation():

    def __init__(self, evaluation_df, video_path, simulation_size) -> None:
        
        # Evaluation dataframe
        self.evaluation_df = evaluation_df

        # Video
        self.video_path = video_path
        self.video = cv2.VideoCapture(video_path)
        self.total_frame_count = min(int(self.video.get(cv2.CAP_PROP_FRAME_COUNT)), evaluation_df.shape[0])

        # DO NOT CHANGE THESE VARIABLES:
        self.deep_racer_fps = 15     # The AWS DeepRacer car takes 15 actions per second, making the animation run at 15 fps
        self.rotate_angle = -90      # Offset angle

        # EXPERIMENT WITH VARIABLES:
        self.frame_rate = 15                 # Variable frame rate to slow-down or speed-up the animation
        self.max_frame_rate = 100            # Maximum variable frame rate allowed
        self.window_w = simulation_size[0]            # Simulation window width
        self.window_h = simulation_size[1]            # Simulation window height
        self.angle_30 = 60                   # Actual angle to display the 30 degrees angle mark in the action space
        self.action_space_radius = 150       # Radius of action space (pixels)
        self.heatmap_attenuation_factor = 10 # Attenuation factor for the heatmap (changes how much weight the current action is given for the overall heatmap)

        # AUXILIARY VARIABLES
        self.last_time_checkpoint = 0                # last time checkpoint (controls the temporal spacing between frames)
        self.frame_number = 0                        # current frame
        self.frame_update = 0                        # controls the update of frames while on pause (0:keep on same frame; 1:advance to next frame; -1:go back to previous frame)
        self.overlay_action_space = True             # boolean to control if action space graphic should be overlayed on top of the simulation
        self.overlay_action_space_heatmap = True     # boolean to control if action space heatmap graphic should be overlayed on top of the simulation (only shown if overlay_action_space==True)
        self.overlay_annotations = True              # boolean to control if text should be overlayed on top of the simulation
        self.quit = False                            # boolean to Quit the simulation upon user's request
        self.pause = False                           # boolean to Pause the simulation upon user's request
        self.show_help = False                       # boolean to show the Help menu (list of commands) upon user's request
        self.key_pressed = ""                        # keeps last keyboard key pressed

        # Initialize the action space heatmap
        self.action_space_heatmap = np.zeros((self.window_h, self.window_w), dtype=np.float64)
        self.action_space_pos = (self.window_w//2+50, 170)    # Position to display the center of the action space. Point of (0m/s, 0 degrees)

        # Define 2 colors for text
        self.black = (0,0,0)
        self.white = (255,255,255)


    # Generate a Gaussian 2D kernel
    def gaussian_filter(self, n, std):
        gaussian_1d = signal.windows.gaussian(n, std)
        gaussian_2d = np.outer(gaussian_1d, gaussian_1d)
        return gaussian_2d

    # Writes text with a different color border (useful for changing backgrounds as in this simulation)
    def write_text_with_border(self, image, text, pos, font, size, color, border_color, thickness, thickness_increase):
        cv2.putText(image, text, pos, font, size, border_color, thickness+thickness_increase)
        cv2.putText(image, text, pos, font, size, color, thickness)

    # Draw the help menu
    def draw_help_menu(self, image):
        h = self.window_h
        w = self.window_w

        margin = 50
        commands_y_offset = 120
        y_grid_space = 20
        cv2.rectangle(image, (margin,margin), (w-margin,h-margin), (255,255,255), -1)

        # Title
        cv2.putText(image, "Commands", (margin+20,margin+40), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,0), 1)

        # List of commands to show
        commands_help = [
            "Q: quit",
            "P: pause/resume",
            "H: help menu",
            "N/M: decrease/increase FPS",
            "A: overlay action space",
            "S: overlay action space heatmap",
            "0-9: jump to point in simulation",
            ",/.: go back / advance one frame (while on pause)",
            "Z/X: decrease/increase heatmap attenuation factor",
            "T: toggle text notes",
            "R: reset heatmap"
        ]
        for idx, command_help in enumerate(commands_help):
            cv2.putText(image, command_help, (margin+20,commands_y_offset + idx*y_grid_space), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1)

    def draw_circle_section(self, image, center, axes, angle, angle_range, color, thickness):
        cv2.ellipse(image, center, axes, angle, angle_range[0], angle_range[1], color, thickness)

    def draw_steer_angle_line(self, image, center, angle, length, color, thickness):
        rad = math.radians(angle)
        pt1 = center
        pt2 = (int(center[0] + length*math.cos(rad)), int(center[1] + length*math.sin(rad)))
        cv2.line(image, pt1, pt2, color, thickness)

    def draw_action(self, image, position, radius, throttle, steer):
        p_x = position[0]
        p_y = position[1]

        x = int(p_x+radius*(throttle/4)*math.cos(math.radians(self.angle_30*steer/30+self.rotate_angle)))
        y = int(p_y+radius*(throttle/4)*math.sin(math.radians(self.angle_30*steer/30+self.rotate_angle)))
        pos = (x, y)
        cv2.circle(image, pos, 5, (0,0,0), -1)

        self.write_text_with_border(image, f"( {throttle}m/s, {steer} )", (p_x+30, p_y+15), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)

    def draw_action_space(self, image, position, radius):
        # Create a black image
        action_space_img = np.zeros(image.shape, np.uint8)

        self.draw_circle_section(action_space_img, position, (radius,radius), self.rotate_angle, (-90,90), self.white, -1)                 # Draw white background
        # Draw circle section for each of the m/s markings
        for i in range(0,4):
            self.draw_circle_section(action_space_img, position, (radius*(4-i)//4,radius*(4-i)//4), self.rotate_angle, (-90,90), self.black, 1)    
        
        # Draw steer angle line and the angle annotations
        for i in range(-2,3):
            self.draw_steer_angle_line(action_space_img, position, self.angle_30*i//2 + self.rotate_angle, radius, self.black, 1)
            x = int(position[0]+radius*math.cos(math.radians(self.angle_30*i//2+self.rotate_angle)))
            if i==0: x-=5
            if i<0: x-=33
            y = int(position[1]+radius*math.sin(math.radians(self.angle_30*i//2+self.rotate_angle)))
            self.write_text_with_border(image, str(30*i//2), (x, y), cv2.FONT_HERSHEY_PLAIN, 1, self.white, self.black, 1, 1)

        # Speed labels
        for i in range(0, 5):
            self.write_text_with_border(image, str(4-i), (position[0]-radius*(4-i)//4-5, position[1]+15), cv2.FONT_HERSHEY_PLAIN, 1, self.white, self.black, 1, 1)

        return action_space_img

    def update_action_space_heatmap(self, throttle, steer):
        p_x = self.action_space_pos[0]
        p_y = self.action_space_pos[1]
        x = int(p_x+self.action_space_radius*(throttle/4)*math.cos(math.radians(self.angle_30*steer/30+self.rotate_angle)))
        y = int(p_y+self.action_space_radius*(throttle/4)*math.sin(math.radians(self.angle_30*steer/30+self.rotate_angle)))

        kernel_size = 80
        std = kernel_size//6       # This standard deviation gives a good smooth curve at the edge
        gaussian = self.gaussian_filter(kernel_size, std)
        gaussian -= gaussian.min()  # Normalize the guassian
        gaussian /= gaussian.max()
        k2 = kernel_size//2
        # Attenuate the effect of the current action
        self.action_space_heatmap[y-k2:y+k2, x-k2:x+k2] += gaussian / self.heatmap_attenuation_factor

        return np.array(self.action_space_heatmap)

    # Display progress bar for the entire simulation at the bottom
    def display_progress_bar(self, image):
        current_x = int(self.window_w * (self.frame_number/self.total_frame_count))
        cv2.rectangle(image, (0, self.window_h-5), (current_x, self.window_h), (0, 255, 0), -1)

    # Update entire simulation
    def update_simulation_display(self, update_action_space_heatmap_b=True):
        
        self.video.set(cv2.CAP_PROP_POS_FRAMES, self.frame_number-1)
        success, frame = self.video.read()
        if not success:
            print(f"Error reading frame {self.frame_number}.")

        if self.show_help:
            self.draw_help_menu(frame)
            cv2.imshow(self.video_path, frame)
            return

        # Resize evaluation video to the specified window size
        frame = cv2.resize(frame, (self.window_w,self. window_h))
        
        times_faster = self.frame_rate / self.deep_racer_fps
        throttle = self.evaluation_df.iloc[self.frame_number].get("throttle")
        steer = -self.evaluation_df.iloc[self.frame_number].get("steer")
        all_wheels_on_track = self.evaluation_df.iloc[self.frame_number].get("all_wheels_on_track")
        progress = self.evaluation_df.iloc[self.frame_number].get("progress")
        pause_left = self.evaluation_df.iloc[self.frame_number].get("pause_duration")
        pos_x = self.evaluation_df.iloc[self.frame_number].get("X")
        pos_y = self.evaluation_df.iloc[self.frame_number].get("Y")
        yaw = self.evaluation_df.iloc[self.frame_number].get("yaw")
        closest_waypoint = self.evaluation_df.iloc[self.frame_number].get("closest_waypoint")
        reward = self.evaluation_df.iloc[self.frame_number].get("reward")
        episode_status = self.evaluation_df.iloc[self.frame_number].get("episode_status")

        # Add text annotations
        if self.overlay_annotations:

            # Track-specific variables:
            y_offset = -30
            if episode_status != "pause":
                self.write_text_with_border(frame, f"State: {episode_status}", (int(self.window_w*0.75), int(self.window_h*0.7+y_offset)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            else:
                self.write_text_with_border(frame, f"State: {episode_status} ({pause_left:.1f}s)", (int(self.window_w*0.75), int(self.window_h*0.7+y_offset)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
                
            self.write_text_with_border(frame, f"Lap progress: {progress:.1f}%", (int(self.window_w*0.75), int(self.window_h*0.7+y_offset+15)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            self.write_text_with_border(frame, f"Closest waypoint: {closest_waypoint}", (int(self.window_w*0.75), int(self.window_h*0.7+y_offset+30)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            
            # Car-specific variables:
            w_perc = 0.02
            h_perc = 0.5
            self.write_text_with_border(frame, "Car variables", (int(self.window_w*w_perc), int(self.window_h*h_perc)), cv2.FONT_HERSHEY_PLAIN, 1, self.white, self.black, 1, 1)
            self.write_text_with_border(frame, f"All wheels on track: {all_wheels_on_track}", (int(self.window_w*w_perc), int(self.window_h*h_perc + 15)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            self.write_text_with_border(frame, f"Position: ({pos_x:.1f}, {pos_y:.1f})", (int(self.window_w*w_perc), int(self.window_h*h_perc + 30)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            self.write_text_with_border(frame, f"Yaw: {yaw:.1f}", (int(self.window_w*w_perc), int(self.window_h*h_perc + 45)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            self.write_text_with_border(frame, f"Reward: {reward:.2f}", (int(self.window_w*w_perc), int(self.window_h*h_perc + 60)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)
            
            # Simulation-specific variables:
            w_perc = 0.02
            h_perc = 0.8
            self.write_text_with_border(frame, "Simulation variables", (int(self.window_w*w_perc), int(self.window_h*h_perc)), cv2.FONT_HERSHEY_PLAIN, 1, self.white, self.black, 1, 1)
            self.write_text_with_border(frame, f"Heatmap attenuation factor: {self.heatmap_attenuation_factor}", (int(self.window_w*w_perc), int(self.window_h*h_perc+15)), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)

            # Print footnote details (FPS, frame count, Help)
            self.write_text_with_border(frame, f"FPS {self.frame_rate} ({times_faster:.1f}x) | Frame {self.frame_number} | H (help) | Q (quit)", (self.window_w//2-10, self.window_h-15), cv2.FONT_HERSHEY_PLAIN, 0.8, self.white, self.black, 1, 1)

        # Add action space
        if self.overlay_action_space:
            action_space_img = self.draw_action_space(frame, self.action_space_pos, self.action_space_radius)
            frame = cv2.addWeighted(frame, 1, action_space_img, 0.8, 1)

            # Add action space heatmap
            if self.overlay_action_space_heatmap:
                # Disable update during pause, not to dissipate the heatmap
                if update_action_space_heatmap_b:
                    action_space_heatmap_copy = self.update_action_space_heatmap(throttle, steer)
                else:
                    action_space_heatmap_copy = np.array(self.action_space_heatmap)
                # Avoid division by zero when reseting heatmap
                if action_space_heatmap_copy.max() > 0:
                    action_space_heatmap_copy *= 255./action_space_heatmap_copy.max()
                # Apply color map
                heatmap = cv2.applyColorMap(np.array(action_space_heatmap_copy, dtype=np.uint8), cv2.COLORMAP_HOT)
                heatmap[heatmap.sum(2) < 120] = 0
                frame[heatmap.sum(2) > 0] = heatmap[heatmap.sum(2) > 0]
                
            self.draw_action(frame, self.action_space_pos, self.action_space_radius, throttle, steer)

        # Reset action space heatmap on states 'off_track' and 'pause'
        if episode_status in ["off_track", "pause"]:
            self.action_space_heatmap = np.zeros((self.window_h, self.window_w), dtype=np.float64) # reset heatmap

        self.display_progress_bar(frame)

        cv2.imshow(self.video_path, frame)


    def listen_for_key_strokes(self):
        # Wait 1 milisecond for a key to be pressed
        self.key_pressed = cv2.waitKey(1)

        if self.key_pressed in [ord('p'), ord('P')]:
            self.pause = not self.pause
        if self.key_pressed in [ord('h'), ord('H')]:
            self.show_help = not self.show_help
            self.pause = self.show_help
        if self.key_pressed in [ord('q'), ord('Q')]:
            self.quit = True
        if self.key_pressed == ord('.'):
            if self.frame_number < self.total_frame_count-1:
                self.frame_update = 1
        if self.key_pressed == ord(','):
            if self.frame_number >= 1:
                self.frame_update = -1
        if self.key_pressed in [ord('z'), ord('Z')]:
            if self.heatmap_attenuation_factor > 1:
                self.heatmap_attenuation_factor -= 1
        if self.key_pressed in [ord('x'), ord('X')]:
            if self.heatmap_attenuation_factor < 50:
                self.heatmap_attenuation_factor += 1
        if self.key_pressed in [ord('r'), ord('R')]:
            self.action_space_heatmap = np.zeros((self.window_h, self.window_w), dtype=np.float64) # reset heatmap
        if self.key_pressed in [ord(str(i)) for i in range(0, 10)]:
            set_progress = int(chr(self.key_pressed))
            # Define the current frame as a percentage. If user clicked on "2" then simulation resumes at 20% of completion
            self.frame_number = int(self.total_frame_count * set_progress/10.)
        if self.key_pressed in [ord('t'), ord('T')]:
            self.overlay_annotations = not self.overlay_annotations

    def listen_for_key_strokes_after_pause(self):
        if self.key_pressed in [ord('n'), ord('N')]:
            if self.frame_rate > 1:
                self.frame_rate -= 1
        if self.key_pressed in [ord('m'), ord('M')]:
            if self.frame_rate < self.max_frame_rate:
                self.frame_rate += 1
        if self.key_pressed in [ord('a'), ord('A')]:
            self.overlay_action_space = not self.overlay_action_space
        if self.key_pressed in [ord('s'), ord('S')]:
            self.overlay_action_space_heatmap = not self.overlay_action_space_heatmap
    

In [23]:
simulation_size = (640, 480)
sim = Simulation(evaluation_df, video_path, simulation_size)

# Run simulation until reaching the last frame
while sim.frame_number < sim.total_frame_count:
    if sim.quit:
        break

    while True:
        # Check for keyboard strokes (user commands)
        sim.listen_for_key_strokes()
        
        if sim.quit:
            break

        if sim.pause:
            if(sim.frame_update == 0):
                sim.update_simulation_display(False)
            else:
                # If during pause, user changes frame, update frame here
                sim.frame_number += sim.frame_update
                sim.frame_update = 0
                sim.update_simulation_display()
            continue

        # These key strokes are not allowed during pause, that's why they follow the pause
        sim.listen_for_key_strokes_after_pause()

        # If enough time has passed (considering the frame rate) since the last frame update, then update again
        time_elapsed = time.time() - sim.last_time_checkpoint
        if time_elapsed > 1./sim.frame_rate:
            sim.last_time_checkpoint = time.time()  # reset time
            
            sim.update_simulation_display()
            break   # break from While(True)
        
    sim.frame_number +=1    # Advance frame

# Present the closing frame:
success, frame = sim.video.read()
sim.write_text_with_border(frame, "Press any key to exit", (30, sim.window_h//2), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255), (0,0,0), 2, 3)
cv2.imshow(video_path, frame)
cv2.waitKey()

# Close video and windows
sim.video.release()
cv2.destroyAllWindows()